# Make graph

In [25]:
file = "src/cond-mat.hg2"

In [26]:
FIN = open(file, "r")

In [27]:
PERIOD1 = 1995
PERIOD2 = 1996
PERIOD3 = 1997
PERIOD4 = 1998
PERIOD5 = 1999
PERIOD6 = 2000

In [28]:
from datetime import tzinfo, timedelta, datetime

ZERO = timedelta(0)
HOUR = timedelta(hours=1)

# A UTC class.

class UTC(tzinfo):
    """UTC"""

    def utcoffset(self, dt):
        return ZERO

    def tzname(self, dt):
        return "UTC"

    def dst(self, dt):
        return ZERO

utc = UTC()

In [29]:
reading_edges = False
import re, datetime

train_set = set()
test_set = set()

def read_edge(line, train_set, test_set):
    m = re.search(r"(\d+) (\d+) \[ (.*) \]", line)
    time = int(m.group(1))
    edgeCount = int(m.group(2))
    edgeList = m.group(3).split()
    is_train = (PERIOD1 <= datetime.datetime.fromtimestamp(time, utc).year <= PERIOD5)
    is_test = (datetime.datetime.fromtimestamp(time, utc).year == PERIOD6)     
    
    for i in range(edgeCount):
        for j in range(i + 1, edgeCount):
            if is_train:
                train_set.add( edgeList[i] + " " + edgeList[j])
            if is_test:
                test_set.add( edgeList[i] + " " + edgeList[j])

for line in FIN:
    if reading_edges:
        read_edge(line, train_set, test_set)
    if "Hyperedges" in line:
        reading_edges = True

In [30]:
len(train_set), len(test_set)

(43601, 21404)

In [31]:
fout_train = open('train.in', 'w')
for line in train_set:
    fout_train.write(line + '\n')
fout_train.close()

fout_test = open('test.out', 'w')
for line in test_set:
    fout_test.write(line + '\n')
fout_test.close()